In [1]:
!git clone https://github.com/cg123/mergekit.git
%cd mergekit
%pip install -e .

fatal: destination path 'mergekit' already exists and is not an empty directory.
mllm/fine_tune/yml_dataset/mergekit
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /system/linux/mambaforge-3.11/lib/python3.11/site-packages/matlabengineforpython-9.14-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Obtaining file://mllm/fine_tune/yml_dataset/mergekit
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mergekit (pyproject.toml) ... done
  Created wheel for mergekit: filename=mergekit-0.0.4.1-0.editable-py3-none-any.whl size=9681 sha256=9dcd95fefbf5bfa8d421f537

In [8]:
OUTPUT_PATH = "./merged_finetune_gpt2_yelp"  # folder to store the result in
LORA_MERGE_CACHE = "/tmp"  # change if you want to keep these for some reason
CONFIG_YML = "mllm/test_merge/fine_tune_yelp_combined.yml"  # merge configuration file
COPY_TOKENIZER = True  # you want a tokenizer? yeah, that's what i thought
LAZY_UNPICKLE = False  # experimental low-memory model loader
LOW_CPU_MEMORY = False  # enable if you somehow have more VRAM than RAM+swap

In [9]:
# actually do merge
import torch
import yaml

from mergekit.config import MergeConfiguration
from mergekit.merge import MergeOptions, run_merge

with open(CONFIG_YML, "r", encoding="utf-8") as fp:
    merge_config = MergeConfiguration.model_validate(yaml.safe_load(fp))

run_merge(
    merge_config,
    out_path=OUTPUT_PATH,
    options=MergeOptions(
        lora_merge_cache=LORA_MERGE_CACHE,
        cuda=torch.cuda.is_available(),
        copy_tokenizer=COPY_TOKENIZER,
        lazy_unpickle=LAZY_UNPICKLE,
        low_cpu_memory=LOW_CPU_MEMORY,
        allow_crimes=True
    ),
)
print("Done!")

100%|██████████| 819/819 [00:03<00:00, 232.36it/s] 


Done!


In [10]:
from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

In [14]:
username = HF_USERNAME

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

\```yaml
{{- yaml_config -}}
\```
"""
yaml_config = """
models:
  - model: yelp_finetune_epoch_1_gpt2_1
    parameters:
      weight: 1.0

  - model: yelp_finetune_epoch_1_gpt2_2
    parameters:
      weight: 1.0

merge_method: linear
dtype: float16
"""
# Create a Jinja template object
jinja_template = Template(template_text.strip())
data = yaml.safe_load(yaml_config)
MODEL_NAME = "Merged_yml"
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

In [15]:
from huggingface_hub import HfApi

username = HF_USERNAME

# Defined in the secrets tab in Google Colab
api = HfApi(token=HF_TOKEN)

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merged_finetune_gpt2_yelp",
)


model-00001-of-00001.safetensors:   0%|          | 0.00/249M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Phudish/Merged_yml/commit/4765047f8a8350192bf10bad9528b4b55dbcfded', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4765047f8a8350192bf10bad9528b4b55dbcfded', pr_url=None, pr_revision=None, pr_num=None)